In [7]:
from rclpy.time import Time as ros2Time
from rclpy.time import Duration

from rosidl_runtime_py.utilities import get_message
from rclpy.serialization import serialize_message, deserialize_message
import rosbag2_py
from irt_nav_msgs.msg import FactorResidual, FactorResiduals, ResidualSample

BAG_PATH = "/home/haoming/Desktop/rosbag2_2024_05_30-09_30_16/"


converter_options = rosbag2_py._storage.ConverterOptions('', '')
bag_reader = rosbag2_py.SequentialReader()
storge_opt = rosbag2_py._storage.StorageOptions(
    uri=BAG_PATH,
    storage_id='mcap'
)
bag_reader.open(storge_opt, converter_options)

residuals = {}

while bag_reader.has_next():
    data_tuple = bag_reader.read_next()
    topic = data_tuple[0]
    data = data_tuple[1]
    ts = data_tuple[2]
    msg = deserialize_message(data, FactorResiduals)
    ts_header = ros2Time(seconds=msg.header.stamp.sec, nanoseconds=msg.header.stamp.nanosec)

    if ts_header.nanoseconds in residuals.keys():
        residuals[ts_header.nanoseconds].append(msg)
    else:
        residuals.update({ts_header.nanoseconds: [msg]})


In [8]:
len(residuals)

276

In [14]:
test = residuals[1606417094926298368]
gp_res = []
for msg in test:
    for res in msg.residuals:
        if res.factor_name == "GPWNOAPriorPose3Factor":
            gp_res.append(res)


In [16]:
len(gp_res[0].samples)

49

In [17]:
gp_res[0].samples[0]

irt_nav_msgs.msg.ResidualSample(id=0, type=1, whitened_error=[0.00027744426104518, 5.375354573807726e-05, 0.0001036285210254097, 0.0008305926248271082, 0.0017616481566364098, 0.001074435962141962, 0.0008141840435186803, -0.0003036548038049891, 0.0001670757358742811, 0.014172511015393956, -0.016159936121480763, 0.8719336279562157], unwhitened_error=[5.6255119082922135e-05, -1.5739744431626965e-05, 1.310041531266851e-05, 0.002675088682507064, -0.0027646950954501176, 0.15930593923503134, 0.0004700697057438564, -0.00017531530543811832, 9.646128860490209e-05, 0.025875364711475335, -0.02950389245796156, 1.591926836610737], loss_error=0.38036803824281157, noise_model_weight=1.0)

In [18]:
gp_res[0].samples[1]

irt_nav_msgs.msg.ResidualSample(id=1, type=3, whitened_error=[-0.026260052986594087, 5.366286157674517e-05, 0.00010325900367855564, 0.0008305838843506996, -0.002315304076779705, 0.0009977660599644889, 0.0008141840443228648, -0.00030365970690611016, 0.00016701041877952347, 0.014172502192909659, -0.018128385125507976, 0.8718968418975482], unwhitened_error=[-0.0008283287416666759, -1.574305031999403e-05, 1.3084326972287449e-05, 0.0026750861504202944, -0.0035538321210903945, 0.15929114132266609, 0.00047006970620815287, -0.000175318136246852, 9.642357773626351e-05, 0.025875348603885184, -0.03309777471635744, 1.591859674716611], loss_error=0.38071551701929557, noise_model_weight=1.0)